In [1]:
from dataclasses import dataclass
import os
import sys

In [2]:
%%capture
import mercury as mr
import pandas as pd
from IPython.display import display

from astropy.time import Time, TimeDelta
from lucupy.minimodel.constraints import CloudCover, ImageQuality
from lucupy.minimodel.semester import SemesterHalf
from lucupy.minimodel.site import Site, ALL_SITES
from lucupy.observatory.abstract import ObservatoryProperties
from lucupy.observatory.gemini import GeminiProperties

from definitions import ROOT_DIR
from scheduler.core.builder.blueprint import CollectorBlueprint, OptimizerBlueprint
from scheduler.core.builder.validationbuilder import ValidationBuilder
from scheduler.core.components.collector import *
from scheduler.core.output import (print_collector_info, plans_table, pickle_plans, pickle_selection)
from scheduler.core.programprovider.ocs import read_ocs_zipfile, OcsProgramProvider
from scheduler.core.statscalculator import StatCalculator

ModuleNotFoundError: No module named 'mercury'

In [3]:
# Add the parent directory to the Python search path.
print(os.getcwd())
cwd = os.getcwd().replace('/demo', '')
sys.path.insert(0, cwd)

# Create a fixed representation of one day.
day = TimeDelta(1.0, format='jd')

# Application parameters.
app = mr.App(title="Scheduler Validation Test",
        description="Try the Validation mode for the Scheduler",
        show_code=False,
        show_prompt=True,
        continuous_update=False,
        static_notebook=False,
        show_sidebar=True,
        full_screen=True,
        allow_download=True)

/Users/sraaphorst/Development/SchedulerRepos/scheduler/demo


NameError: name 'TimeDelta' is not defined

In [ ]:
out_dir = mr.OutputDir()
print(f"Output directory path: {out_dir.path}")

In [ ]:
# Set up the UI.
start_date = mr.Text(label='Start Date', value='2018-10-01')
schedule_nights = mr.Numeric(label='Nights to Schedule', value=1, min=1, max=365, step=1)
viscalc_nights = mr.Numeric(label='Nights for Visibility Calculations', value=1, min=1, max=365, step=1)

site_choices = [site.name for site in Site]
site_selection = mr.MultiSelect(label="Sites",
                                choices=site_choices,
                                value=site_choices)

cc_choices = [cc.name for cc in CloudCover]
cc_selection = mr.Select(label='Cloud Cover',
                         choices=cc_choices,
                         value=CloudCover.CC50.name)

iq_choices = [iq.name for iq in ImageQuality]
iq_selection = mr.Select(label='Image Quality',
                         choices=iq_choices,
                         value=ImageQuality.IQ70.name)


@dataclass(frozen=True)
class Params:
    start: Time
    end: Time
    num_nights_to_schedule: int
    sites: FrozenSet[Site]
    cc: CloudCover
    iq: ImageQuality
       
def extract_params() -> Params:
    # Parse the start time for the visibility calculations.
    start_string = f'{start_date.value} 08:00:00'
    try:
        # Since we are using UT, add one day to the start date as it will roll back.
        start = Time(start_string, format='iso', scale='utc') + day
    except Exception as e:
        print(f'Could not parse the start date "{start_date.value}".')
        raise mr.StopExecution
        
    # The end time for the visibility calculations.
    # In the case that the end time == start time, we get one night.
    num_nights_visibility = viscalc_nights.value
    if num_nights_visibility < 1 or int(num_nights_visibility) != num_nights_visibility:
        print(f'The number of visibility nights must be a positive integer: "{num_nights_visibility}".')
        raise mr.StopExecution
    num_nights_visibility = int(num_nights_visibility)
    end = start + (num_nights_visibility - 1) * day
    
    num_nights_to_schedule = schedule_nights.value
    if num_nights_to_schedule < 1 or int(num_nights_to_schedule) != num_nights_to_schedule:
        print(f'The number of nights to schedule must be a positive integer: "{num_nights_to_schedule}".')
        raise mr.StopExecution
    if num_nights_to_schedule > num_nights_visibility:
        print(f'The number of nights to schedule must be at most the number of nights used in visibility calculations.')
        raise mr.StopExecution
    num_nights_to_schedule = int(num_nights_to_schedule)
    
    sites = frozenset(Site[s] for s in site_selection.value)
    if not sites:
        print('No sites specified.')
        raise mr.StopExecution
    
    try:
        cc = CloudCover[cc_selection.value]
    except KeyError:
        print(f'Illegal CloudCover: "{cc_selection.value}"')
        raise mr.StopExecution
    
    try:
        iq = ImageQuality[iq_selection.value]
    except KeyError:
        print(f'Illegal ImageQuality: "{iq_selection.value}"')
        raise mr.StopExecution
    
    return Params(start=start,
                  end=end,
                  num_nights_to_schedule=num_nights_to_schedule,
                  sites=sites,
                  cc=cc,
                  iq=iq)


## SCHEDULER 

When the Scheduler is running, the UI will be frozen until the results are displayed.
When the run button is enabled, the Scheduler may be invoked.
The default values provided are the ones used during internal testing.

In [ ]:
# Validate the UI data.
params = extract_params()

print('Parameters parsed: Scheduler execution running...\n')

# If validation is successful, we run the Scheduler.
ObservatoryProperties.set_properties(GeminiProperties)

# Read in a list of JSON data
programs = read_ocs_zipfile(os.path.join(ROOT_DIR, 'scheduler', 'data', '2018B_program_samples.zip'))

# Create the Collector and load the programs.
collector_blueprint = CollectorBlueprint(
    ['SCIENCE', 'PROGCAL', 'PARTNERCAL'],
    ['Q', 'LP', 'FT', 'DD'],
    1.0
)

builder = None
try:
    builder = ValidationBuilder(Sources())
except TypeError as ex:
    import inspect
    sig = inspect.signature(ValidationBuilder.__init__)
    print(f'ValidationBuilder signature: {sig}')
    print(f'TypeError: {ex}')
    class_file_path = inspect.getfile(ValidationBuilder)
    print(f'ValidationBuilder is being loaded from {class_file_path}')

if builder is None:
    print('ValidationBuilder fail.')
    sys.exit(1)

collector = builder.build_collector(
    start=params.start,
    end=params.end,
    sites=params.sites,
    semesters=frozenset([Semester(2018, SemesterHalf.B)]),
    blueprint=collector_blueprint
)

selector = builder.build_selector(collector,
                                  num_nights_to_schedule=params.num_nights_to_schedule,
                                  default_cc=params.cc,
                                  default_iq=params.iq)

# Prepare the optimizer.
optimizer_blueprint = OptimizerBlueprint(
    "GreedyMax"
)
optimizer = builder.build_optimizer(
    blueprint=optimizer_blueprint
)

# The total nights for which visibility calculations have been done.
total_nights = len(collector.time_grid)

# Create the overall plans by night.
overall_plans = {}

for night_idx in range(selector.num_nights_to_schedule):
    # We score one night at a time.
    night_indices = np.array([night_idx])
    
    # Retrieve the Selection and run the Optimizer to get the plans.
    selection = selector.select(night_indices=night_indices)
    pickle_selection(selection, out_dir.path, night_idx)
    plans = optimizer.schedule(selection)
    night_plans = plans[0]

    # Store the plans in the overall_plans array for that night.
    # TODO: This might be an issue. We may need to index nights (plans) in optimizer by night_idx.
    overall_plans[night_idx] = night_plans

    # Perform the time accounting on the plans.
    collector.time_accounting(night_plans)


overall_plans = [p for _, p in sorted(overall_plans.items())]
plan_summary = StatCalculator.calculate_plans_stats(overall_plans, collector)

night_tables = plans_table(overall_plans)
pickle_plans(night_tables, out_dir.path, params.start, params.end)

print('\n\n\n')
for n_idx, table_per_site in enumerate(night_tables):
    night_date = (params.start + (n_idx - 1) * day).iso.split(' ')[0]
    mr.Md(f"## Night: {night_date} (Day: {n_idx})")
    for site in params.sites:
        night_stats = overall_plans[n_idx][site].night_stats
        mr.Md(f"### Site: {site.value[0]} ")
        display(table_per_site[site])
        mr.Md(f'Time loss: {night_stats.time_loss}')
        mr.Md(f'Plan_Score: {night_stats.plan_score}')
        mr.Md(f'Number of ToOs: {night_stats.n_toos}')
        mr.Md(f'Schedule Observations by band')
        for key, value in night_stats.completion_fraction.items():
            mr.Md(f'Band {key:<10}{value}')
    mr.Md('---')            
mr.Md('## Program Completion')
df_pcompl = pd.DataFrame(plan_summary).T.rename(columns={0: "% Completion", 1: "Score"})
display(df_pcompl)

print('DONE')


In [ ]:
import inspect
sig = inspect.signature(ValidationBuilder.__init__)
print(sig)